In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#pd.set_option('display.max_rows', None) 

In [11]:
df=pd.read_csv('./food.csv',sep='\t',low_memory=False)

In [32]:
df['categories_fr'].isnull().sum()/len(df)

0.7368504732333246

In [3]:
df.shape ## 320772 lignes et 172 colonnes

(320772, 162)

# # Traitements des colonnes

In [20]:
## fonction renvoyant les colonnes et le pourcentage de valeurs nulles pour chacune d'elle
def columns_na_percentage(df):
    na_df=(df.isnull().sum()/len(df)*100).sort_values(ascending=False).reset_index()
    na_df.columns = ['Column','na_rate_percent']
    return na_df
  

In [24]:
## colonnes comportant trop de valeurs nulles
na_df=columns_na_percentage(df)
na_columns=na_df[na_df['na_rate_percent']>=50] ## 128 colonnes ont plus de 50% de valeurs nulles
#na_columns[na_columns['Column']=='manufacturing_places']
na_columns

AttributeError: 'Series' object has no attribute 'columns'

In [6]:
## suppression des colonnes ayant plus 50% de valeurs nulles
df.drop(columns=na_columns['Column'].tolist(),axis=1,inplace=True)

In [8]:
## le nom du produit censé être unique et non null
## suppression de toutes les lignes dont le nom du produit est non renseigné ou nul
mask_to_remove=df[(pd.isna(df['product_name']))|(df['product_name'].str.strip()=='')]
len(mask_to_remove['product_name'] )##17765 lignes à supprimmer
#df=df[~pd.isna(df['product_name'])]

48        NaN
183       NaN
187       NaN
227       NaN
233       NaN
         ... 
320747    NaN
320748    NaN
320752    NaN
320755    NaN
320766    NaN
Name: product_name, Length: 17765, dtype: object

In [15]:
## suppression des produits dont les noms ne sont pas renseignés
df.drop(mask_to_remove.index,inplace=True)

In [16]:
## vérification de doublons pour le nom du produit 106092 doublons
df[df['product_name'].duplicated(keep=False)][['product_name','last_modified_t']]

,product_name,last_modified_t
0,Farine de blé noir,1474103893
2,Peanuts,1489069957
4,Organic Polenta,1489055653
6,Organic Long Grain White Rice,1489055730
7,Organic Muesli,1489055712
...,...,...
320753,Lager,1444561971
320754,nf test,1472253428
320757,Amandes,1491079939
320760,Poireaux,1460898286


In [17]:
## suppression des doublons sur les produits
df.drop_duplicates(subset='product_name',keep='last',inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221346 entries, 1 to 320771
Data columns (total 34 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   code                                     221342 non-null  object 
 1   url                                      221342 non-null  object 
 2   creator                                  221345 non-null  object 
 3   created_t                                221346 non-null  object 
 4   created_datetime                         221342 non-null  object 
 5   last_modified_t                          221346 non-null  object 
 6   last_modified_datetime                   221346 non-null  object 
 7   product_name                             221346 non-null  object 
 8   brands                                   210532 non-null  object 
 9   brands_tags                              210526 non-null  object 
 10  countries                       

In [19]:
## identification des colonnes décrivant la part nutritionnelle pour 100g
nutriment_cols=[i for i in df.columns.tolist() if i.endswith('_100g')]
nutriment_cols

['energy_100g',
 'fat_100g',
 'saturated-fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'proteins_100g',
 'salt_100g',
 'sodium_100g',
 'nutrition-score-fr_100g',
 'nutrition-score-uk_100g']

In [20]:
all_missing_nutritional = df[df["energy_100g"].isna()
                            & df["proteins_100g"].isna()
                            & df["sugars_100g"].isna()
                            & df["fat_100g"].isna()
            &df['sugars_100g'].isna()&
 df['fiber_100g'].isna() & df[ 'nutrition-score-fr_100g'].isna()& df['nutrition-score-uk_100g'].isna() 
                            & df["saturated-fat_100g"].isna()
                            & df["carbohydrates_100g"].isna()
                            & df["sodium_100g"].isna() 
                    ] 
all_missing_nutritional[nutriment_cols].head() ## 35727

,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
## suppression des colonnes dont toutes les valeurs nutritives sont inconnues ainsi que les nutriscores
df.drop(all_missing_nutritional.index,inplace=True)



In [16]:
## les colonnes code, url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime
  # sont peu utiles, et plus relatives au stockage en base de données

## les variables 'nutrition-score-uk_100g' et 'nutrition-score-fr_100g' sont identiques
## les variables countries et states, sont des concaténation de pays ce qui peut s'avérer très complexe
## Au final, on ne garde que les colonnes des valeurs nutritives des produits et leur nutriscore
cols_to_remove=['code','url','creator','created_t',
                'created_datetime','last_modified_t',
                'last_modified_datetime','nutrition-score-uk_100g',
                'countries','countries_tags',
                'countries_fr',
                'brands',
                'brands_tags',
                'ingredients_text'
               
               ]

#df['nutrition-score-uk_100g'].isnull().sum()==df['nutrition-score-fr_100g'].isnull().sum()


df=df[nutriment_cols]
df.drop('nutrition-score-uk_100g',axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185619 entries, 1 to 320771
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   energy_100g              184296 non-null  float64
 1   fat_100g                 169736 non-null  float64
 2   saturated-fat_100g       162893 non-null  float64
 3   carbohydrates_100g       169458 non-null  float64
 4   sugars_100g              173535 non-null  float64
 5   fiber_100g               142059 non-null  float64
 6   proteins_100g            183398 non-null  float64
 7   salt_100g                179943 non-null  float64
 8   sodium_100g              179912 non-null  float64
 9   nutrition-score-fr_100g  157577 non-null  float64
dtypes: float64(10)
memory usage: 15.6 MB


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185619 entries, 1 to 320771
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   energy_100g              184296 non-null  float64
 1   fat_100g                 169736 non-null  float64
 2   saturated-fat_100g       162893 non-null  float64
 3   carbohydrates_100g       169458 non-null  float64
 4   sugars_100g              173535 non-null  float64
 5   fiber_100g               142059 non-null  float64
 6   proteins_100g            183398 non-null  float64
 7   salt_100g                179943 non-null  float64
 8   sodium_100g              179912 non-null  float64
 9   nutrition-score-fr_100g  157577 non-null  float64
dtypes: float64(10)
memory usage: 15.6 MB


In [7]:
from pandas.plotting import scatter_matrix
def clean_df(df):
    ## colonnes comportant trop de valeurs nulles
    na_df=columns_na_percentage(df)
    na_columns=na_df[na_df['na_rate_percent']>=50] ## 128 colonnes ont plus de 50% de valeurs nulles
#na_columns[na_columns['Column']=='manufacturing_places']
    
    ## suppression des colonnes ayant plus 50% de valeurs nulles
    df.drop(columns=na_columns['Column'].tolist(),axis=1,inplace=True)
    ## le nom du produit censé être unique et non null
    ## suppression de toutes les lignes dont le nom du produit est non renseigné ou nul
    mask_to_remove=df[(pd.isna(df['product_name']))|(df['product_name'].str.strip()=='')]
    len(mask_to_remove['product_name']) ##17765 lignes à supprimmer
    #df=df[~pd.isna(df['product_name'])]
    ## suppression des produits dont les noms ne sont pas renseignés
    df.drop(mask_to_remove.index,inplace=True)
    ## vérification de doublons pour le nom du produit 106092 doublons
    df[df['product_name'].duplicated(keep=False)][['product_name','last_modified_t']]
    ## suppression des doublons sur les produits
    df.drop_duplicates(subset='product_name',keep='last',inplace=True)
    ## identification des colonnes décrivant la part nutritionnelle pour 100g
    nutriment_cols=[i for i in df.columns.tolist() if i.endswith('_100g')]
    
    all_missing_nutritional = df[df["energy_100g"].isna()
                            & df["proteins_100g"].isna()
                            & df["sugars_100g"].isna()
                            & df["fat_100g"].isna()
            &df['sugars_100g'].isna()&
     df['fiber_100g'].isna() & df[ 'nutrition-score-fr_100g'].isna()& df['nutrition-score-uk_100g'].isna() 
                            & df["saturated-fat_100g"].isna()
                            & df["carbohydrates_100g"].isna()
                            & df["sodium_100g"].isna() 
                    ] 
    all_missing_nutritional[nutriment_cols].head() ## 35727
    ## suppression des colonnes dont toutes les valeurs nutritives sont inconnues ainsi que les nutriscores
    df.drop(all_missing_nutritional.index,inplace=True)
    
        ## les colonnes code, url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime
      # sont peu utiles, et plus relatives au stockage en base de données

    ## les variables 'nutrition-score-uk_100g' et 'nutrition-score-fr_100g' sont identiques
    ## les variables countries et states, sont des concaténation de pays ce qui peut s'avérer très complexe
    ## Au final, on ne garde que les colonnes des valeurs nutritives des produits et leur nutriscore et le nombre d'additifs présent
    cols_to_remove=['code','url','creator','created_t',
                    'created_datetime','last_modified_t',
                    'last_modified_datetime','nutrition-score-uk_100g',
                    'countries',
                    'countries_tags',
                    'countries_fr',
                    'brands',
                    'brands_tags',
                    'ingredients_text'

                   ]

    #df['nutrition-score-uk_100g'].isnull().sum()==df['nutrition-score-fr_100g'].isnull().sum()

    nutriment_cols.remove('nutrition-score-uk_100g')
    relevant_cols = nutriment_cols
    #relevant_cols.append('additives_n')
    relevant_cols.append('product_name')
    
    print('relevant_cols',relevant_cols)
    df=df[relevant_cols]
    
    df=df.set_index('product_name')
    df.to_csv('./cleaned_food.csv')
    df.info()


In [8]:
clean_df(df)

relevant_cols ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g', 'nutrition-score-fr_100g', 'product_name']
<class 'pandas.core.frame.DataFrame'>
Index: 185619 entries, Banana Chips Sweetened (Whole) to Sugar Free Drink Mix, Peach Tea
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   energy_100g              184296 non-null  float64
 1   fat_100g                 169736 non-null  float64
 2   saturated-fat_100g       162893 non-null  float64
 3   carbohydrates_100g       169458 non-null  float64
 4   sugars_100g              173535 non-null  float64
 5   fiber_100g               142059 non-null  float64
 6   proteins_100g            183398 non-null  float64
 7   salt_100g                179943 non-null  float64
 8   sodium_100g              179912 non-null  float64
 9   nutrition-score-fr_100g  15